In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

def phases(iq: np.ndarray,start:int):
    list = []
    for i in range(start,len(iq)-2,6):
        list.append(
            np.angle(complex(iq[i],iq[i+1]))
        )
    return np.array(list)



antenna_conf= "2-2-0-5-6"

data = pd.read_json("./folder/data.json")
data = data[data["Switch Order"] == antenna_conf]
#data = data[data["TAG"] != "B"]
#data = data[data["rssi"] >-70]
del data["level_0"]
del data["index"]
data=data.reset_index()


,index,timestamp,rssi,frequency,iq,Switch Order,Angle,Antenna,TAG,Entrada,phase_0,phase_5,phase_6,fases_50,fases_60,fases_65
0,9096,2024-08-22 15:16:16.275000+00:00,-67,2426,"[-142, -137, 120, -126, 126, 139, -167, 104, -...",2-2-0-5-6,0,1,A,1,"[-3.1244023699890597, 0.29201954061992613, -2....","[-2.339560165086297, 1.2942492071802545, -1.48...","[-2.494920308070028, 1.040504177328581, -1.707...","[0.7848422049027626, 1.0022296665603285, 0.839...","[0.6294820619190316, 0.7484846367086551, 0.615...","[0.15536014298373102, 0.25374502985167346, 0.2..."
1,9097,2024-08-22 15:16:16.779000+00:00,-58,2402,"[-80, 141, -131, -103, 101, -130, 118, 114, -1...",2-2-0-5-6,0,1,A,1,"[2.5894396631342893, -0.17380111908918636, -2....","[2.5262531117844014, -0.34500217720710513, -3....","[2.7041780923641787, -0.02547219651072259, -2....","[0.06318655134988793, 0.17120105811791886, 0.3...","[0.11473842922988942, 0.14832892257846364, 0.0...","[0.17792498057977735, 0.3195299806963825, 0.29..."
2,9098,2024-08-22 15:16:17.292000+00:00,-62,2480,"[-191, 41, -39, -193, 189, -37, 18, 189, -200,...",2-2-0-5-6,0,1,A,1,"[2.2157118618019473, -0.44099776135465124, 3.0...","[2.918561685218235, 0.20333608402827574, -2.40...","[-2.4756234162106834, 1.1441688336680205, -1.5...","[0.7028498234162877, 0.6443338453829268, 0.845...","[1.591850029166956, 1.585166595022672, 1.68069...","[0.8890002057506674, 0.9408327496397444, 0.835..."
3,9099,2024-08-22 15:16:17.797000+00:00,-65,2426,"[-124, 130, -114, -138, 137, -130, 113, 136, -...",2-2-0-5-6,0,1,A,1,"[1.5459787281772228, -1.1410340476982082, 2.46...","[2.4309448208736515, -0.31213546609693965, -3....","[2.2930467110880133, -0.4903474940060718, 3.08...","[0.8849660926964287, 0.8288985816012686, 0.790...","[0.7470679829107905, 0.6506865536921365, 0.619...","[0.1378981097856382, 0.17821202790913215, 0.17..."
4,9100,2024-08-22 15:16:18.251000+00:00,-61,2480,"[-181, -66, 63, -189, 179, 69, -84, 168, -152,...",2-2-0-5-6,0,1,A,1,"[2.8305943729842524, 0.1959811336361145, -2.68...","[-2.7297853482280767, 0.8404605587711186, -1.8...","[-1.7590178320996674, 1.5042281630190728, -1.1...","[0.7228055859672571, 0.6444794251350041, 0.794...","[1.693573102095666, 1.308247029382958, 1.48933...","[0.9707675161284097, 0.6637676042479543, 0.694..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9861,18957,2024-08-23 10:24:15.812000+00:00,-55,2402,"[180, -64, 5, 186, -196, -9, 31, -183, 171, 88...",2-2-0-5-6,90,2,C,2,"[3.141592653589793, 0.9588938923602299, -1.311...","[0.6479307586687755, -1.554038119212659, 2.602...","[-2.1995926132103296, 1.8249807137550402, -0.3...","[2.4936618949210176, 2.512932011572889, 2.3693...","[0.9420000403794635, 0.8660868213948101, 0.996...","[2.847523371879105, 2.9041664742118867, 2.9176..."
9862,18958,2024-08-23 10:24:16.305000+00:00,-56,2426,"[148, 115, -134, 131, -116, -141, 163, -84, 63...",2-2-0-5-6,90,2,C,2,"[-2.1763409903998667, 1.8055189392800937, -0.3...","[1.413120336369888, -0.6212445105300607, -3.00...","[-1.3351024479214708, 2.7682907627771343, 0.48...","[2.693723980409832, 2.4267634498101542, 2.6930...","[0.8412385424783961, 0.9627718234970404, 0.802...","[2.748222784291359, 2.893650033872391, 2.78744..."
9863,18959,2024-08-23 10:24:16.820000+00:00,-62,2480,"[-64, 153, -151, -87, 113, -133, 125, 117, -13...",2-2-0-5-6,90,2,C,2,"[-0.9532040533357405, -3.026716048172894, 0.90...","[2.8037144653495147, 0.6128202021652414, -1.59...","[0.28297689337281756, -1.968670684412066, 2.25...","[2.526266788494331, 2.6436490568414506, 2.4925...","[1.2361809467085578, 1.0580453637608276, 1.357...","[2.520737571976697, 2.581490886577307, 2.43349..."
9864,18960,2024-08-23 10:24:17.334000+00:00,-63,2480,"[-155, -6, 6, -159, 158, 64, -86, 160, -110, -...",2-2-0-5-6,90,2,C,2,"[0.5841785304099635, -1.7008277031365882, 2.18...","[-2.2477111786373736, 2.0971886910213167, -0.1...","[1.6156380418737804, -0.6534263411807619, -2.6...","[2.831889709047337, 2.485168913021681, 2.34145...","[1.03145951

# On IQ samples

In [2]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

#Loading the Iris dataset



X, y = data["iq"].to_list(), data["Angle"].map(lambda x: {"0":0,
                                                "30":1,
                                                "60":2,
                                                "90":3,
                                                "120":4,
                                                "150":5,
                                                "180":6,
                                                }.get(str(x))).to_list()

#Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#Creating an XGBoost classifier
model = xgb.XGBClassifier(enable_categorical=True)

#Training the model on the training data
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [3]:
#Making predictions on the test set
predictions = model.predict(X_test)

#Calculating accuracy
accuracy = accuracy_score(y_test, predictions)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, predictions, target_names={"0","30","60","90","120","150","180"}))

Accuracy: 0.9797297297297297

Classification Report:
              precision    recall  f1-score   support

          30       0.97      0.99      0.98       434
           0       0.99      0.97      0.98       408
         180       0.98      1.00      0.99       452
          60       0.98      0.98      0.98       412
         150       0.98      0.95      0.96       425
         120       0.96      0.99      0.98       416
          90       0.99      0.99      0.99       413

    accuracy                           0.98      2960
   macro avg       0.98      0.98      0.98      2960
weighted avg       0.98      0.98      0.98      2960



In [4]:
data.iloc[0]["iq"]
model.predict([data.iloc[0]["iq"]])

array([0])

In [7]:
import paho.mqtt.client as mqtt
import json
import numpy as np
import datetime


# Callback chamado quando o cliente recebe a resposta CONNACK do servidor.
def on_connect(client, userdata, flags, rc):
    print("Conectado com o código de resultado "+str(rc))
    client.subscribe("/gw/ac233fc09283/status")
    
# Callback chamado quando uma mensagem PUBLISH é recebida do servidor.
def on_message(client, userdata, msg):
    data = json.loads(msg.payload)
    prediction = model.predict([data[1]["aoa"]["iq"]])
    print({ "0":0,
            "1":30,
            "2":60,
            "3":90,
            "4":120,
            "5":150,
            "6":180}.get(str(prediction[0])))

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("broker.hivemq.com", 1883, 60)

now = datetime.datetime.now()
client.loop_forever()


Conectado com o código de resultado 0
90
90
90
60
90
90
90
120
120
150
120
120
120
120
150
120
120
150
120
120
120
150
120
120
120
120
120
150
120
150
120
120
150
120
150
120
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
120
90
90
90
120
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
120
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
120
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
120
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
120
90
90
120
90
90
90
90
120
90
90
90
120
90
90
90
90
120
90
90
90
90
90
90
90

KeyboardInterrupt: 

# Phase diffs  ( no reference )

In [5]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

#Loading the Iris dataset



X, y = data["iq"].map(lambda y: [ np.angle(complex(y[i-1], y[i])) for i in range(16,len(y),2)]).to_list(), data["Angle"].map(lambda x: {"0":0,
                                                "30":1,
                                                "60":2,
                                                "90":3,
                                                "120":4,
                                                "150":5,
                                                "180":6,
                                                }.get(str(x))).to_list()

#Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#Creating an XGBoost classifier
model = xgb.XGBClassifier(enable_categorical=True)

#Training the model on the training data
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [6]:
#Making predictions on the test set
predictions = model.predict(X_test)

#Calculating accuracy
accuracy = accuracy_score(y_test, predictions)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, predictions, target_names={"0","30","60","90","120","150","180"}))

Accuracy: 0.9658783783783784

Classification Report:
              precision    recall  f1-score   support

         120       0.96      0.96      0.96       437
          90       0.95      0.98      0.96       385
          30       0.99      0.99      0.99       419
         180       0.97      1.00      0.98       440
         150       0.98      0.92      0.95       419
           0       0.95      0.97      0.96       444
          60       0.97      0.95      0.96       416

    accuracy                           0.97      2960
   macro avg       0.97      0.97      0.97      2960
weighted avg       0.97      0.97      0.97      2960



In [8]:
data["iq"].iloc[0].__len__()

84